<a href="https://colab.research.google.com/github/venkateshchandle/bis/blob/main/ant_colony_tsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random
import math
import numpy as np

class AntColonyTSP:
    def __init__(self, cities, n_ants=50, n_iterations=1000, alpha=1, beta=5, rho=0.1, Q=100):
        self.cities = cities
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        self.n_cities = len(cities)

        self.pheromone = np.ones((self.n_cities, self.n_cities)) / self.n_cities
        self.distance_matrix = self.calculate_distance_matrix()

    def calculate_distance_matrix(self):
        dist_matrix = np.zeros((self.n_cities, self.n_cities))
        for i in range(self.n_cities):
            for j in range(i + 1, self.n_cities):
                dist = math.sqrt((self.cities[i][0] - self.cities[j][0]) ** 2 +
                                 (self.cities[i][1] - self.cities[j][1]) ** 2)
                dist_matrix[i][j] = dist_matrix[j][i] = dist
        return dist_matrix

    def choose_next_city(self, ant, visited):
        current_city = ant[-1]
        probabilities = []
        total_pheromone = 0.0

        for city in range(self.n_cities):
            if city not in visited:
                pheromone = self.pheromone[current_city][city] ** self.alpha
                distance = self.distance_matrix[current_city][city] ** self.beta
                total_pheromone += pheromone * (1 / distance)

        for city in range(self.n_cities):
            if city not in visited:
                pheromone = self.pheromone[current_city][city] ** self.alpha
                distance = self.distance_matrix[current_city][city] ** self.beta
                probability = (pheromone * (1 / distance)) / total_pheromone
                probabilities.append((city, probability))


        next_city = self.select_city(probabilities)
        return next_city

    def select_city(self, probabilities):
        """ Select city based on roulette wheel selection (probabilities). """
        rand = random.random()
        cumulative_prob = 0.0
        for city, prob in probabilities:
            cumulative_prob += prob
            if cumulative_prob >= rand:
                return city
        return probabilities[-1][0]

    def construct_solution(self):
        solution = []
        visited = set()
        start_city = random.randint(0, self.n_cities - 1)
        solution.append(start_city)
        visited.add(start_city)

        while len(solution) < self.n_cities:
            next_city = self.choose_next_city(solution, visited)
            solution.append(next_city)
            visited.add(next_city)

        solution.append(solution[0])
        return solution

    def calculate_solution_length(self, solution):
        length = 0.0
        for i in range(len(solution) - 1):
            length += self.distance_matrix[solution[i]][solution[i + 1]]
        return length

    def update_pheromones(self, ants):
        # Pheromone evaporation
        self.pheromone *= (1 - self.rho)

        for ant in ants:
            solution_length = self.calculate_solution_length(ant)
            pheromone_deposit = self.Q / solution_length

            for i in range(len(ant) - 1):
                self.pheromone[ant[i]][ant[i + 1]] += pheromone_deposit

    def solve(self):
        best_solution = None
        best_length = float('inf')

        for iteration in range(self.n_iterations):
            ants = [self.construct_solution() for _ in range(self.n_ants)]
            for ant in ants:
                solution_length = self.calculate_solution_length(ant)
                if solution_length < best_length:
                    best_solution = ant
                    best_length = solution_length

            self.update_pheromones(ants)

            if (iteration + 1) % 100 == 0:
                print(f"Iteration {iteration + 1}: Best Length = {best_length:.2f}")

        return best_solution, best_length

cities = [
    (0, 0),  # City 0
    (1, 2),  # City 1
    (4, 3),  # City 2
    (6, 4),  # City 3
    (5, 0)   # City 4
]

aco = AntColonyTSP(cities, n_ants=20, n_iterations=500, alpha=1, beta=2, rho=0.1, Q=100)
best_solution, best_length = aco.solve()

print("Best Solution:", best_solution)
print("Best Length:", best_length)


Iteration 100: Best Length = 16.76
Iteration 200: Best Length = 16.76
Iteration 300: Best Length = 16.76
Iteration 400: Best Length = 16.76
Iteration 500: Best Length = 16.76
Best Solution: [1, 2, 3, 4, 0, 1]
Best Length: 16.75751924078562
